<a href="https://colab.research.google.com/github/VorobyvEgor/Seminar_Sber/blob/main/Seminars/%D0%A1%D0%B5%D0%BC%D0%B8%D0%BD%D0%B0%D1%80_4_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.decomposition import PCA

import os

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

sns.set(font_scale=1.3)

red = '#FF3300'
blue = '#0099CC'
green = '#00CC66'

# Метод главных компонент
### (Principal component analysis, PCA)

Интерактивная визуализация PCA и собственных векторов:

* <a href="http://setosa.io/ev/principal-component-analysis/">Principal Component Analysis</a>

* <a href="http://setosa.io/ev/eigenvectors-and-eigenvalues/">Eigenvectors and Eigenvalues</a>

## 1. SVD-разложение

Генерируем датасет из равномерного распределения.

In [ ]:
X = np.random.normal(size=(100, 10))
print(X.shape)

In [ ]:
pd.DataFrame(X)

Вот так можно вычислять сингулярное разложение этого датасета.

In [ ]:
U, D, V = sp.linalg.svd(X, full_matrices=False)  # If True (default), U and V are of shape (M, M), (N, N). If False, the shapes are (M, K) and (K, N), where K = min(M, N).
print(U.shape, D.shape, V.shape)

Проверим, что получилось. Восстановим датасет и сравним с исходной версией.

In [ ]:
# Посмотрим, что из себя представляет матрица D

print(...)

In [ ]:
# Напомним как создать единичную матрицу

print(...)

In [ ]:
D * np.eye(10)

In [ ]:
X_restored = np.dot(U, np.dot(D * np.eye(10), V))
print(np.allclose(X_restored, X))

True


Восстановленная версия действительно очень близка к исходной.

## 2. PCA

Реализация из sklearn:

<a href="https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA">`sklearn.decomposition.PCA`</a>`(n_components=None)`

Методы класса:
* `fit(X)` — обучиться на данных `X`;
* `fit_transforn(X)` — обучиться на данных `X` и вернуть сжатое представление `X`;
* `transform(X_new)` — вернуть сжатое представление `X_new` для обученной ранее модели;
* `inverse_transform(Y)` — восстановить сжатые данные `Y` в исходное пространство.

Атрибуты класса:

* `components_` — главные компоненты в порядке убывания собственных чисел, размер (n_components, n_features);
* `explained_variance_` — дисперсия вдоль главных компонент, равны собственным числам, размер (n_components,);
* `explained_variance_ratio_` —- доля дисперсии, объясняемая каждой компонентой, размер (n_components,);
* `mean_` — среднее по данным, размер (n_components,);
* `noise_variance_` — оценка дисперсии шума для метода Probabilistic PCA.

Другие модификации, реализованные в sklearn:

* <a href="https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.KernelPCA">`KernelPCA`</a>;
* <a href="https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.SparsePCA">`SparsePCA`</a>;
* <a href="https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.IncrementalPCA">`IncrementalPCA`</a>.

Генерируем двумерный датасет из нормального распределения.

In [ ]:
X = np.random.multivariate_normal(size=150, mean=[0, 3], cov=[[3, 1], [1, 1]])
# X = np.random.multivariate_normal(size=150, mean=[0, 3, 4, 5], cov=[[3, 1, 2, 1], [1, 1, 1, 1], [3, 4, 9, 1], [3, 1, 3, 5]])

In [ ]:
X.shape

In [ ]:
pd.DataFrame(X)

In [ ]:
pca = PCA(n_components=1)
Y = pca.fit_transform(X)
X_hat = pca.inverse_transform(Y)

In [ ]:
Y.shape

In [ ]:
X_hat.shape

Его главные компоненты (точнее, одна компонента) — двумерные векторы.

In [ ]:
pca.components_

Вектор средних:

In [ ]:
pca.mean_

На первом графике синим отмечены исходные точки, красным - они же после проецирования и обратного преобразования.
На втором графике точки в одномерном пространстве.

In [ ]:
pd.DataFrame(X)

In [ ]:
pd.DataFrame(X_hat)

In [ ]:
plt.figure(figsize=(12, 7))
plt.scatter(X[:, 0], X[:, 1], alpha=0.7, color=blue)
plt.scatter(X_hat[:, 0], X_hat[:, 1], color=red, alpha=0.7)
plt.xlabel('Исходный признак 1')
plt.ylabel('Исходный признак 2')
plt.axis('equal')
plt.show()

plt.figure(figsize=(10, 1))
plt.scatter(Y, np.zeros(len(Y)), alpha=0.5, color=red)
plt.xlabel('Проекция на первую главную компоненту')
plt.show()

Другие методы снижения размерности:

* http://scikit-learn.org/stable/modules/manifold.html#manifold

Примеры с визуализацией:

* http://scikit-learn.org/stable/auto_examples/manifold/plot_compare_methods.html#sphx-glr-auto-examples-manifold-plot-compare-methods-py

* http://scikit-learn.org/stable/auto_examples/manifold/plot_manifold_sphere.html#sphx-glr-auto-examples-manifold-plot-manifold-sphere-py

## 3. Cжатие изображений с помощью PCA

In [ ]:
! wget https://www.dropbox.com/s/ehhrw5l46rpnv61/3840x2400.png

In [ ]:
! ls

In [ ]:
image = plt.imread('3840x2400.png')
image.shape

In [ ]:
# посмотрим что из себя представляет image

...

In [ ]:
plt.imshow(image)

Применим к картинке преобразование для выделения 24 блоков размера 600x640.

In [ ]:
# Хотим: 4 * 6 блока по (600, 640, 3). То есть массив размерности (24, 600, 640, 3
image.shape

In [ ]:
image.transpose((1, 2, 0)).shape

In [ ]:
# Разбиваем высоту

image.transpose((1, 2, 0)).reshape((3840, 3, 4, 600)).shape 

In [ ]:
image.transpose((1, 2, 0)).reshape((3840, 3, 4, 600)).transpose((1, 2, 3, 0)).shape 

In [ ]:
# Разбиваем ширину

image.transpose((1, 2, 0)).reshape((3840, 3, 4, 600)).transpose((1, 2, 3, 0)).reshape((3, 4, 600, 6, 640)).shape 

In [ ]:
image.transpose((1, 2, 0)).reshape((3840, 3, 4, 600)).transpose((1, 2, 3, 0)).reshape((3, 4, 600, 6, 640)).transpose((1, 3, 2, 4, 0)).shape 

In [ ]:
# Схлопываем

image.transpose((1, 2, 0)).reshape((3840, 3, 4, 600)).transpose((1, 2, 3, 0)).reshape((3, 4, 600, 6, 640)).transpose((1, 3, 2, 4, 0)).reshape((4 * 6, 600 * 640 * 3)).shape 

In [ ]:
# Итого:

X = image.transpose((1, 2, 0)).reshape((3840, 3, 4, 600)) \
         .transpose((1, 2, 3, 0)).reshape((3, 4, 600, 6, 640)) \
         .transpose((1, 3, 2, 4, 0)).reshape((4 * 6, 600 * 640 * 3))
print(X.shape)

In [ ]:
# Визуализируем блоки.

plt.figure(figsize=(11, 7))
for i in range(24):
    plt.subplot(4, 6, i + 1)
    plt.imshow(X[i].reshape((600, 640, 3)))
    plt.axis('off');

# Как видим, все правильно.

Напишем несколько функций.

In [ ]:
def draw_components(pca, n, m):
    """
    pca - обученная PCA модель
    n - количество горизонтальных блоков картинки
    m - количество вертикальных блоков картинки
    """
    print('Среднее изображение')
    plt.figure(figsize=(1, 1))
    plt.imshow(pca.mean_.reshape((n, m, 3)), cmap='gray')
    plt.axis('off')
    plt.show()

    print('Главные компоненты')
    plt.figure(figsize=(11, len(pca.components_) // 10 + 1))
    for i, comp in enumerate(pca.components_):
        plt.subplot(len(pca.components_) // 10 + 1, 10, i + 1)
        img = pca.components_[i].reshape((n, m, 3))
        plt.imshow((img - img.min()) / (img.max() - img.min()), cmap='gray')
        plt.axis('off')
    plt.show()

        
def image_pca(image, n, m, n_components=20, draw_picture=True, 
              draw_comp=True, visualization=True):
    """
    image - исходаня картинка
    n - количество горизонтальных блоков картинки
    m - количество вертикальных блоков картинки
    n_components - количество главных компонент
    draw_picture - показывать ли исходную картинку
    draw_comp - рисовать ли главные компоненты
    visualization - рисовать ли проекцию на первые три компоненты
    """

    # Показываем исходную картинку
    if draw_picture:
        plt.figure(figsize=(15, 7))
        plt.imshow(image)
        plt.axis('off')
        plt.show()
    print("Размерность оригинальной картинки: ", image.shape)
    
    # Разбиение на блоки
    N, M, K = image.shape
    X = image.transpose((1, 2, 0)).reshape((M, K, N // n, n)) \
             .transpose((1, 2, 3, 0)).reshape((K, N // n, n, M // m, m)) \
             .transpose((1, 3, 2, 4, 0)).reshape((N * M // (n * m), n * m * K))
    X = X.reshape((N * M // (n * m), n * m * K))
    
    
    # Применение PCA
    pca = PCA(n_components=n_components)
    Y = pca.fit_transform(X)
    X_hat = pca.inverse_transform(Y)
    
    # Разбираемся с интенсивностью цвета
    max_value = X.max()
    X_hat = X_hat * (X_hat <= max_value) + max_value * (X_hat > max_value)
    X_hat = X_hat * (X_hat >= 0)
    
    # Собираем картинку из блоков
    X_hat = X_hat.reshape((N // n, M // m, n, m, K)).transpose((1, 3, 4, 0, 2))\
                 .reshape((M // m, m, K, N)).transpose((3, 2, 0, 1))\
                 .reshape((N, K, M)).transpose((0, 2, 1))
    
    # Рисуем восстановленную картинку
    plt.figure(figsize=(15, 7))
    plt.imshow(X_hat)
    plt.axis('off')
    plt.title('Восстановленное изображение при {} главных компонентах.\n'.format(n_components))
    plt.show()
    print("Размерность сжатой картинки: ", X_hat.shape)
    
    # Если нужно, рисуем главные компоненты 
    if draw_comp:
        draw_components(pca, n, m)
    
    # Визуализируем проекцию на первые три компоненты (третья - цвет)
    if visualization:
        pca = PCA(n_components=3)
        Y = pca.fit_transform(X)
        plt.figure(figsize=(15, 10))
        plt.scatter(Y[:, 0], Y[:, 1], c=Y[:, 2], alpha=0.1)
        plt.xlabel('Проекция на первую главную компоненту')
        plt.ylabel('Проекция на вторую главную компоненту')
        plt.title('Проекция на первые три компоненты (третья - цвет)')
        plt.show()
    plt.imsave('im1.png', image)
    print("Размер оригинального файла", os.path.getsize('im1.png'))

    plt.imsave('im2.png', X_hat)
    print("Размер сжатого файла", os.path.getsize('im2.png'))

In [ ]:
image = plt.imread('3840x2400.png')
image_pca(image, 10, 10)

In [ ]:
for n_components in [15, 10, 5, 4, 3, 2, 1]:
    image_pca(image, 10, 10, 
              n_components=n_components, draw_picture=False,
              draw_comp=False, visualization=False)

## 4. PCA для MNIST

In [ ]:
! wget https://www.dropbox.com/s/gq1tj9bzj8dkcul/mnist_train.csv

In [ ]:
mnist_data_all = pd.read_csv('gdrive/MyDrive/DS_class/5_LIN_AL/mnist_train.csv')

In [ ]:
mnist_label = mnist_data_all['label']

mnist_data = mnist_data_all.drop(columns = ["label"])

In [ ]:
print(mnist_data.shape)
print(mnist_label.shape)

In [ ]:
mnist_data

In [ ]:
idx = 6

print("Label: ", mnist_label[idx])

plt.figure(figsize=(7,7))

grid_data = mnist_data.iloc[idx].to_numpy().reshape(28,28)  # reshape from 1d to 2d pixel array
plt.imshow(grid_data, interpolation = "none", cmap = "gray")
plt.show()

Каждое число - 784 фичи. Как нам одним взглядом посмотреть на весь датасет?

In [ ]:
pca = PCA(n_components=2)

In [ ]:
pca.fit(mnist_data)

In [ ]:
mnist_data_2D = pca.transform(mnist_data)

In [ ]:
mnist_data_2D.shape

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(mnist_data_2D[:,0], mnist_data_2D[:,1], c=mnist_label)
plt.colorbar()
plt.show()

### Проклятие размерности
Большая размерность приводит к следующим проблемам

*   Нужно много памяти
*   Трудоемкие вычисления
*   Все элементы выборки начинают находится примерно на одинаковом расстоянии друг от друга!



In [ ]:
# Продемострируем проклятье размерности

...

Давайте как проклятье размерности может повлиять на работу алгоритмов машинного обучения

In [ ]:
! wget https://www.dropbox.com/s/xuya93ez6ff712x/mnist_test.csv?dl=0

In [ ]:
mnist_data_test_all = pd.read_csv('gdrive/MyDrive/DS_class/5_LIN_AL/mnist_test.csv')

In [ ]:
mnist_label_test = mnist_data_test_all['label']

mnist_data_test = mnist_data_test_all.drop(columns = ["label"])

In [ ]:
mnist_label_test.shape

In [ ]:
mnist_data_test

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
def reduce_and_learn(dim=2):
    pca = PCA(n_components=dim)
    pca.fit(mnist_data)
    mnist_data_dim_D = pca.transform(mnist_data)
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(mnist_data_dim_D, mnist_label)

    mnist_data_test_dim_D = pca.transform(mnist_data_test)

    predict_quality = knn.score(mnist_data_test_dim_D, mnist_label_test)
    return predict_quality

In [ ]:
dims = [2, 5, 10, 20, 40, 80, 160, 320, 640, 784]
predicts_quality = []

for dim in dims:
    print(dim)
    predicts_quality.append(reduce_and_learn(dim=dim))

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(dims, predicts_quality)
plt.rc('font', size=12)
plt.xlabel('Размерность')
plt.ylabel('Качество предсказания')
plt.grid()
plt.show()